In [ ]:
# ================================================
# Section 1: Imports and Configuration
# ================================================
import requests
import pandas as pd
from datetime import datetime
import os
import xml.etree.ElementTree as ET
import time
from datetime import timedelta

# API Keys (Replace with your actual values)
PORTCALL_API_KEYS = {
    "US": "your_us_api_key_here",
    "SEA": "your_sea_api_key_here",
    "ES": "your_spain_api_key_here",
    "UK": "your_uk_api_key_here",
    "CN": "your_china_api_key_here",
    # Add more country codes and keys as needed
}

# Power Tools API Keys and Fleet ID
CLEARFLEET_API_KEY = ""
SETFLEET_API_KEY = ""
FLEET_ID = # Add your fleet ID here

# New API key for fleet portcalls (Step 4)
FLEET_PORTCALL_API_KEY = "" #add your fleet portcall api key here

# Set API endpoints and keys
GRAPHQL_URL = "https://api.kpler.marinetraffic.com/v2/vessels/graphql"
GRAPHQL_API_KEY = "" # Add your GraphQL API key here

MARKET_OPTIONS = [
    "CONTAINER SHIPS", "DRY BREAKBULK", "DRY BULK", "LNG CARRIERS",
    "LPG CARRIERS", "WET BULK", "PASSENGER SHIPS", "OFFSHORE/RIGS",
    "RO/RO", "SUPPORTING VESSELS", "PLEASURE CRAFT", "FISHING", "OTHER MARKETS"
]

In [ ]:
# ================================================
# Section 2: Helper Input Functions
# ================================================
def safe_input(prompt, validate_func=None, allowed_values=None):
    while True:
        user_input = input(prompt).strip()
        if user_input.upper() == 'ESC':
            raise KeyboardInterrupt("User pressed ESC - exiting.")
        if not user_input:
            print("Input cannot be empty. Please try again or type ESC to exit.")
            continue
        if allowed_values and user_input.upper() not in allowed_values:
            print(f"Invalid option. Allowed: {', '.join(allowed_values)}")
            continue
        if validate_func:
            try:
                return validate_func(user_input)
            except Exception as e:
                print(f"Invalid input: {e}")
                continue
        else:
            return user_input

def get_user_portcall_input():
    allowed_regions = set(PORTCALL_API_KEYS.keys())
    region = safe_input(f"Select region/country code {sorted(allowed_regions)}: ", allowed_values=allowed_regions).upper()
    
    def validate_date(d): return datetime.strptime(d, "%Y-%m-%d")
    fromdate = safe_input("Enter fromdate (YYYY-MM-DD): ", validate_func=validate_date)
    todate = safe_input("Enter todate (YYYY-MM-DD): ", validate_func=validate_date)
    
    from_date = fromdate.strftime('%Y-%m-%d 00:00:00')
    to_date = todate.strftime('%Y-%m-%d 23:59:00')
    
    print("\nAvailable Market Types:")
    for i, m in enumerate(MARKET_OPTIONS, 1): print(f"{i}. {m}")
    
    def validate_markets(m):
        indices = [int(i.strip()) - 1 for i in m.split(",")]
        return [MARKET_OPTIONS[i] for i in indices]
    
    market_list = safe_input("Select market(s) by number: ", validate_func=validate_markets)
    movetype = 0 # or allow user input as before
    
    return region, from_date, to_date, market_list, movetype

def split_date_range(start_date, end_date, max_days=190):
    """
    Splits a date range into chunks of max_days.
    Returns a list of (chunk_start, chunk_end) tuples.
    """
    chunks = []
    current_start = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    while current_start < end_date:
        current_end = min(current_start + timedelta(days=max_days-1), end_date)
        chunks.append((current_start.strftime('%Y-%m-%d %H:%M:%S'), current_end.strftime('%Y-%m-%d %H:%M:%S')))
        current_start = current_end + timedelta(seconds=1)
    return chunks

In [ ]:
# ================================================
# Section 3: PortCall API Functions
# ================================================
def build_portcall_url(api_key, fromdate, todate, msgtype='extended', market=None, movetype=0):
    url = (
        f"https://services.marinetraffic.com/api/portcalls/{api_key}/v:6/"
        f"fromdate:{fromdate.replace(' ', '%20')}/"
        f"todate:{todate.replace(' ', '%20')}/"
        f"msgtype:{msgtype}/protocol:jsono"
    )
    if market: url += f"/market:{market.replace(' ', '%20')}"
    if movetype is not None: url += f"/movetype:{movetype}"
    return url

def fetch_portcall_data_single(region, from_date, to_date, market_list, movetype):
    api_key = PORTCALL_API_KEYS[region]
    all_data = []
    for market in market_list:
        if movetype in [0, 1]:
            url = build_portcall_url(api_key, from_date, to_date, market=market, movetype=movetype)
            all_data.extend(requests.get(url).json())
        else:
            for mt in [0, 1]:
                url = build_portcall_url(api_key, from_date, to_date, market=market, movetype=mt)
                all_data.extend(requests.get(url).json())
    return all_data

def fetch_portcall_data(region, from_date, to_date, market_list, movetype):
    all_data = []
    date_chunks = split_date_range(from_date, to_date, max_days=190)
    for chunk_start, chunk_end in date_chunks:
        print(f"Fetching port calls from {chunk_start} to {chunk_end}...")
        data = fetch_portcall_data_single(region, chunk_start, chunk_end, market_list, movetype)
        if data:
            all_data.extend(data)
    return all_data

def normalize_portcall_data(data):
    df = pd.json_normalize(data)
    status_map = {0: "NA", 1: "In Ballast", 2: "Partially Laden", 3: "Fully Laden"}
    df = df.rename(columns={
        'IMO': 'IMO', 'SHIPNAME': 'SHIPNAME', 'TIMESTAMP_UTC': 'TIMESTAMP_UTC',
        'MOVE_TYPE': 'MOVE_TYPE', 'PORT_ID': 'PORT_ID', 'PORT_NAME': 'PORT_NAME',
        'SHIP_ID': 'SHIP_ID', 'LOAD_STATUS': 'LOAD_STATUS'
    })
    df['LOAD_STATUS'] = df['LOAD_STATUS'].fillna(0).astype(int).map(status_map).fillna("Unknown")
    # Exclude anchorages and canals
    df = df[~df['PORT_NAME'].str.lower().str.contains('canal|anch', na=False)]
    return df[['IMO', 'SHIPNAME', 'TIMESTAMP_UTC', 'MOVE_TYPE', 'PORT_ID', 'PORT_NAME', 'SHIP_ID', 'LOAD_STATUS']]

def save_df_to_csv(df, prefix="portcalls"):
    filename = f"{prefix}_{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"
    df.to_csv(filename, index=False)
    return filename

In [ ]:
# ================================================
# Section 4: ClearFleet API (Clears fleet list)
# ================================================
def build_clearfleet_url(api_key, fleet_id):
    """Build URL for ClearFleet API."""
    return f"https://services.marinetraffic.com/api/clearfleet/{api_key}/fleet_id:{fleet_id}"

def is_clearfleet_successful(xml_text):
    """Parse XML response and return True if fleet cleared successfully."""
    try:
        root = ET.fromstring(xml_text)
        fleet = root.find(".//FLEET")
        if fleet is not None and fleet.attrib.get("DELETE") == "1":
            return True
        return False
    except ET.ParseError:
        return False

def clear_fleet(api_key, fleet_id):
    """Call ClearFleet API to clear a fleet. Returns success status."""
    url = build_clearfleet_url(api_key, fleet_id)
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        if is_clearfleet_successful(resp.text):
            print("✅ Fleet cleared successfully.")
            return True
        else:
            print(f"❌ Unexpected response:\n{resp.text}")
            return False
    except Exception as e:
        print(f"❌ Failed to clear fleet: {e}")
        return False

In [ ]:
# ================================================
# Section 5: SetFleet API – Add IMOs One-by-One
# ================================================
def build_setfleet_batch_url(api_key, fleet_id, imo_list):
    """Build the SetFleet API URL to add multiple vessels by IMO (batch)."""
    imo_str = ",".join(str(imo) for imo in imo_list)
    return f"https://services.marinetraffic.com/api/setfleet/{api_key}/fleet_id:{fleet_id}/imo:{imo_str}/active:1"

def add_vessels_to_fleet(api_key, fleet_id, imo_list, batch_size=300):
    """Add vessels to a MarineTraffic fleet in batches using IMO. Returns success & failed lists."""
    if not imo_list:
        print("No IMOs provided to add to fleet.")
        return [], []

    success_ids = []
    failed_ids = []

    print(f"\n== Step 3: SetFleet (Add Vessels in Batches of {batch_size}) ===")
    print(f"Adding {len(imo_list)} vessels to fleet {fleet_id}...\n")

    for batch_num, start in enumerate(range(0, len(imo_list), batch_size), 1):
        batch = imo_list[start:start+batch_size]
        url = build_setfleet_batch_url(api_key, fleet_id, batch)
        try:
            resp = requests.get(url, timeout=30)
            resp.raise_for_status()
            root = ET.fromstring(resp.text)
            # Find all ERROR nodes
            error_nodes = root.findall(".//ERROR")
            error_imos = set()
            for error in error_nodes:
                imo = error.attrib.get("IMO")
                error_imos.add(imo)
                print(f"[Batch {batch_num}] ❌ IMO {imo} failed. Error: {error.attrib.get('DESCRIPTION', '')}")
                failed_ids.append((imo, error.attrib.get("DESCRIPTION", "")))
            # Mark all other IMOs as success
            for imo in batch:
                if imo not in error_imos:
                    print(f"[Batch {batch_num}] ✅ IMO {imo} added.")
                    success_ids.append(imo)
        except Exception as e:
            print(f"[Batch {batch_num}] ❌ Batch error: {e}")
            for imo in batch:
                failed_ids.append((imo, str(e)))
        time.sleep(5)  # Add 5-second buffer between API calls

    print(f"\nAdded {len(success_ids)} vessels successfully; {len(failed_ids)} failed.")
    return success_ids, failed_ids

In [ ]:
# ================================================
# Section 6: Step 4 - Fleet Port Call API (Filtered All Records)
# ================================================
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

MAX_DURATION_DAYS = 190

def build_fleet_portcall_url(api_key, fromdate, todate):
    return (
        f"https://services.marinetraffic.com/api/portcalls/{api_key}/v:6/"
        f"fromdate:{fromdate.replace(' ', '%20')}/"
        f"todate:{todate.replace(' ', '%20')}/"
        f"msgtype:extended/protocol:jsono"
    )

def fetch_fleet_portcall_data(api_key, fromdate, todate):
    url = build_fleet_portcall_url(api_key, fromdate, todate)
    resp = requests.get(url)
    resp.raise_for_status()
    return resp.json()

def normalize_fleet_portcalls(data):
    df = pd.json_normalize(data)
    if df.empty:
        return df
    df = df.rename(columns={
        'IMO': 'IMO',
        'SHIPNAME': 'SHIPNAME',
        'TIMESTAMP_UTC': 'TIMESTAMP_UTC',
        'MOVE_TYPE': 'MOVE_TYPE',
        'PORT_ID': 'PORT_ID',
        'PORT_NAME': 'PORT_NAME',
        'PORT_COUNTRY_CODE': 'PORT_COUNTRY_CODE',
        'SHIP_ID': 'SHIP_ID'
    })
    df['TIMESTAMP_UTC'] = pd.to_datetime(df['TIMESTAMP_UTC'])
    # Exclude anchorages and canals
    df = df[~df['PORT_NAME'].str.lower().str.contains('canal|anch', na=False)]
    return df[['IMO', 'SHIPNAME', 'SHIP_ID', 'TIMESTAMP_UTC', 'MOVE_TYPE', 'PORT_ID', 'PORT_NAME', 'PORT_COUNTRY_CODE']]

def save_fleet_portcalls_single(api_key, fromdate_fleet, todate_fleet, region=None):
    all_data = []
    current_start = fromdate_fleet
    while current_start < todate_fleet:
        current_end = min(current_start + timedelta(days=MAX_DURATION_DAYS), todate_fleet)
        print(f"Fetching fleet port calls for region {region} from {current_start} to {current_end}...")
        data = fetch_fleet_portcall_data(
            api_key,
            current_start.strftime('%Y-%m-%d %H:%M:%S'),
            current_end.strftime('%Y-%m-%d %H:%M:%S')
        )
        all_data.extend(data)
        current_start = current_end + timedelta(seconds=1)
    df_fleet = normalize_fleet_portcalls(all_data)
    return df_fleet

def save_fleet_portcalls(api_key, fromdate, todate, region=None):
    region_str = region.lower() if region else "unknown"
    print(f"\n=== Step 4: Fleet Port Call API - All Records for {region_str.upper()} (Excl. region, Departures Only) ===")
    fromdate_fleet = datetime.strptime(fromdate, '%Y-%m-%d %H:%M:%S') - relativedelta(months=2)
    todate_fleet = datetime.strptime(todate, '%Y-%m-%d %H:%M:%S') + relativedelta(months=1)

    all_dfs = []
    date_chunks = split_date_range(fromdate_fleet, todate_fleet, max_days=MAX_DURATION_DAYS)
    for chunk_start, chunk_end in date_chunks:
        df = save_fleet_portcalls_single(api_key, pd.to_datetime(chunk_start), pd.to_datetime(chunk_end), region=region_str)
        if df is not None and not df.empty:
            all_dfs.append(df)
    if all_dfs:
        df_full = pd.concat(all_dfs, ignore_index=True)
        filename = f"{region_str}_fleet_portcalls_filtered_{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"
        df_full.to_csv(filename, index=False)
        print(f"✅ Saved fleet port calls to {filename}")
        return filename
    else:
        print(f"No fleet port call data found for region {region_str}.")
        return None

In [ ]:
# ================================================
# Section 7A: Last Departure & Next Arrival (multi-country, robust)
# ================================================

# Define exclusion lists for each region/country code
EXCLUDE_COUNTRIES = {
    "US": ["US"],
    "SEA": ["SG", "MY", "ID", "TH", "VN", "PH", "KH", "MM", "BN", "LA", "TL"],
    "ES": ["ES"],
    "UK": ["GB", "UK"],
    "CN": ["CN"],
    # Add more country codes and their exclusions as needed
}

def enrich_portcalls_with_last_departure(portcalls_df, fleet_df):
    """
    Enriches portcalls_df with last non-region departure info from fleet_df.
    Adds LAST_PORT_ID, LAST_PORT, LAST_PORT_COUNTRY_CODE, LAST_PORT_TIMESTAMP columns.
    """
    portcalls_df = portcalls_df.copy()
    fleet_df = fleet_df.copy()

    portcalls_df["LAST_PORT_ID"] = pd.NA
    portcalls_df["LAST_PORT"] = pd.NA
    portcalls_df["LAST_PORT_COUNTRY_CODE"] = pd.NA
    portcalls_df["LAST_PORT_TIMESTAMP"] = pd.NaT

    fleet_df["TIMESTAMP_UTC"] = pd.to_datetime(fleet_df["TIMESTAMP_UTC"], errors='coerce', utc=True)
    portcalls_df["TIMESTAMP_UTC"] = pd.to_datetime(portcalls_df["TIMESTAMP_UTC"], errors='coerce', utc=True)

    group_col = "SHIP_ID" if "SHIP_ID" in portcalls_df.columns and "SHIP_ID" in fleet_df.columns else "IMO"
    portcalls_df[group_col] = portcalls_df[group_col].astype(str)
    fleet_df[group_col] = fleet_df[group_col].astype(str)
    fleet_grouped = fleet_df.groupby(group_col)

    for idx, row in portcalls_df.iterrows():
        ship_key = str(row[group_col])
        call_time = row["TIMESTAMP_UTC"]

        if ship_key in fleet_grouped.groups:
            vessel_calls = fleet_grouped.get_group(ship_key)
            prior_departures = vessel_calls[
                (vessel_calls["MOVE_TYPE"] == 1) &
                (vessel_calls["TIMESTAMP_UTC"] < call_time)
            ]
            if not prior_departures.empty:
                latest_dep = prior_departures.sort_values("TIMESTAMP_UTC").iloc[-1]
                portcalls_df.loc[idx, "LAST_PORT_ID"] = latest_dep.get("PORT_ID", pd.NA)
                portcalls_df.loc[idx, "LAST_PORT"] = latest_dep.get("PORT_NAME", pd.NA)
                portcalls_df.loc[idx, "LAST_PORT_COUNTRY_CODE"] = latest_dep.get("PORT_COUNTRY_CODE", pd.NA)
                portcalls_df.loc[idx, "LAST_PORT_TIMESTAMP"] = pd.to_datetime(latest_dep["TIMESTAMP_UTC"]).tz_localize(None)

    return portcalls_df

def enrich_portcalls_with_last_departure_region(portcalls_df, fleet_df, region):
    """
    Filters fleet_df to exclude region ports, then enriches portcalls_df with last departure info.
    """
    region = region.strip().upper()
    exclude_countries = EXCLUDE_COUNTRIES.get(region, [])
    fleet_df = fleet_df.copy()
    if exclude_countries and 'PORT_COUNTRY_CODE' in fleet_df.columns:
        fleet_df = fleet_df[
            ~fleet_df['PORT_COUNTRY_CODE'].str.upper().isin(exclude_countries)
        ]
    return enrich_portcalls_with_last_departure(portcalls_df, fleet_df)

def enrich_portcalls_with_next_nonregion_arrival(portcalls_df, fleet_df, region):
    """
    Enriches portcalls_df with next non-region arrival info from fleet_df.
    Adds NEXT_PORT_ID, NEXT_PORT, NEXT_PORT_COUNTRY, NEXT_PORT_TIMESTAMP columns.
    """
    portcalls_df = portcalls_df.copy()
    fleet_df = fleet_df.copy()

    portcalls_df["TIMESTAMP_UTC"] = pd.to_datetime(portcalls_df["TIMESTAMP_UTC"], utc=True)
    fleet_df["TIMESTAMP_UTC"] = pd.to_datetime(fleet_df["TIMESTAMP_UTC"], utc=True)

    region = region.strip().upper()
    exclude_countries = EXCLUDE_COUNTRIES.get(region, [])

    # Filter out arrivals in the region and only arrivals (MOVE_TYPE == 0)
    df_arrivals = fleet_df[
        (~fleet_df["PORT_COUNTRY_CODE"].str.upper().isin(exclude_countries)) &
        (fleet_df["MOVE_TYPE"] == 0)
    ].copy()

    portcalls_df["NEXT_PORT_ID"] = pd.NA
    portcalls_df["NEXT_PORT"] = pd.NA
    portcalls_df["NEXT_PORT_COUNTRY"] = pd.NA
    portcalls_df["NEXT_PORT_TIMESTAMP"] = pd.NaT

    # Use SHIP_ID for grouping if available, else fallback to IMO
    group_col = "SHIP_ID" if "SHIP_ID" in portcalls_df.columns and "SHIP_ID" in df_arrivals.columns else "IMO"
    portcalls_df[group_col] = portcalls_df[group_col].astype(str)
    df_arrivals[group_col] = df_arrivals[group_col].astype(str)
    arrivals_grouped = df_arrivals.groupby(group_col)

    for idx, row in portcalls_df.iterrows():
        ship_key = str(row[group_col])
        call_time = row["TIMESTAMP_UTC"]

        if ship_key in arrivals_grouped.groups:
            vessel_arrivals = arrivals_grouped.get_group(ship_key)
            future_calls = vessel_arrivals[vessel_arrivals["TIMESTAMP_UTC"] > call_time]
            if not future_calls.empty:
                next_arrival = future_calls.sort_values("TIMESTAMP_UTC").iloc[0]
                portcalls_df.loc[idx, "NEXT_PORT_ID"] = next_arrival.get("PORT_ID", pd.NA)
                portcalls_df.loc[idx, "NEXT_PORT"] = next_arrival.get("PORT_NAME", pd.NA)
                portcalls_df.loc[idx, "NEXT_PORT_COUNTRY"] = next_arrival.get("PORT_COUNTRY_CODE", pd.NA)
                portcalls_df.loc[idx, "NEXT_PORT_TIMESTAMP"] = pd.to_datetime(next_arrival["TIMESTAMP_UTC"]).tz_localize(None)

    return portcalls_df

def enrich_portcalls_with_last_and_next(portcalls_df, fleet_df, region):
    """
    Runs both last departure and next arrival enrichment for the selected region.
    """
    df = enrich_portcalls_with_last_departure_region(portcalls_df, fleet_df, region)
    df = enrich_portcalls_with_next_nonregion_arrival(df, fleet_df, region)
    return df

In [ ]:
# ================================================
# Section 8: Fetch TEU/CAPACITY from MT GraphQL API
# ================================================
def fetch_teu_by_imo_list(imo_list, batch_size=500):
    """
    Fetch teuCapacity for a list of IMOs using the Kpler GraphQL API.
    Returns a DataFrame with columns: IMO, TEU_CAPACITY
    """
    results = []
    for i in range(0, len(imo_list), batch_size):
        batch = imo_list[i:i+batch_size]
        query = f"""
        query Vessels {{
            vessels(
                first: {len(batch)}
                where: {{
                    filters: [
                        {{
                            field: "identifier.imo"
                            op: IN
                            values: [{','.join(f'"{imo}"' for imo in batch)}]
                        }}
                    ]
                    operator: OR
                }}
            ) {{
                nodes {{
                    identifier {{ imo }}
                    particulars {{
                        capacity {{ teuCapacity }}
                    }}
                }}
            }}
        }}
        """

        headers = {
            "Authorization": f"Basic {GRAPHQL_API_KEY}",
            "Content-Type": "application/json"
        }

        response = requests.post(GRAPHQL_URL, json={"query": query}, headers=headers)
        if response.status_code != 200:
            print(f"Error {response.status_code}: {response.text}")
            continue

        data = response.json()
        vessels = data.get("data", {}).get("vessels", {}).get("nodes", [])
        for vessel in vessels:
            imo = vessel.get("identifier", {}).get("imo")
            teu = None
            particulars = vessel.get("particulars", {})
            if particulars and particulars.get("capacity"):
                teu = particulars["capacity"].get("teuCapacity")
            results.append({"IMO": imo, "TEU/CAPACITY": teu})

    import pandas as pd
    return pd.DataFrame(results)

In [ ]:
# ================================================
# Section 9a: Aggregation of Total Arrivals / Vessel Count to US Ports / TEU/capacity Totals
# ================================================
def aggregate_total_port_arrivals(df_enriched, output_filename=None):
    """
    Aggregates total arrivals, unique vessel count, and TEU/capacity totals by PORT_ID, PORT_NAME, and MONTH_YEAR.
    """
    # Ensure TIMESTAMP_UTC is datetime
    df_enriched["TIMESTAMP_UTC"] = pd.to_datetime(df_enriched["TIMESTAMP_UTC"], errors='coerce', dayfirst=True)
    df_enriched["MONTH_YEAR"] = df_enriched["TIMESTAMP_UTC"].dt.strftime('%Y-%m')

    agg = df_enriched.groupby(['PORT_ID', 'PORT_NAME', 'MONTH_YEAR']).agg(
        total_arrivals=('SHIP_ID', 'count'),
        vessel_count=('IMO', 'nunique'),
        teu_capacity_total=('TEU/CAPACITY', lambda x: pd.to_numeric(x, errors='coerce').sum())
    ).reset_index()

    agg['teu_capacity_total'] = agg['teu_capacity_total'].astype('Int64')

    if output_filename:
        agg.to_csv(output_filename, index=False)
        print(f"Aggregation saved to {output_filename}")

    return agg

In [ ]:
# ================================================
# Section 10: Main Function
# ================================================
def main():
    print("MarineTraffic Port Calls + Fleet Management Tool")
    print("Aggregation from existing enriched file.\n")

    try:
        # Step 1: User input for Port Calls (any region)
        region, from_date, to_date, market_list, movetype = get_user_portcall_input()
        region = region.strip().upper()  # Normalize region value

        # Step 2: Fetch Port Call Data (batched if needed)
        print("\n=== Step 2: Fetch Port Call Data ===")
        data = fetch_portcall_data(region, from_date, to_date, market_list, movetype)
        if not data:
            print("No port call data returned.")
            return

        df_portcalls = normalize_portcall_data(data)
        print(f"Fetched {len(df_portcalls)} port call records.")
        portcalls_csv = save_df_to_csv(df_portcalls)
        print(f"✅ Port calls saved to {portcalls_csv}")

        # Step 3: Extract unique IMOs for adding to fleet
        imo_list = df_portcalls['IMO'].dropna().astype(str).unique().tolist()

        # Step 4: Clear Fleet (TEMPORARILY DISABLED)
        print("\n=== Step 4: Clear Fleet ===")
        cleared = clear_fleet(CLEARFLEET_API_KEY, FLEET_ID)
        if not cleared:
            print("Fleet clearing failed. Aborting.")
            return

        # Step 5: Add vessels to Fleet (TEMPORARILY DISABLED)
        add_vessels_to_fleet(SETFLEET_API_KEY, FLEET_ID, imo_list)

        # Step 6: Fetch Fleet Port Calls (batched if needed) and enrich
        fleet_filename = save_fleet_portcalls(FLEET_PORTCALL_API_KEY, from_date, to_date, region=region)
        if not fleet_filename:
            print("No fleet data to enrich port calls.")
            return
        df_fleet_portcalls = pd.read_csv(fleet_filename)

        # Enrich with last non-region departure and next non-region arrival
        df_enriched = enrich_portcalls_with_last_and_next(df_portcalls, df_fleet_portcalls, region)
        print(f"Enriched {len(df_enriched)} port call records.")

        # Step 7: Fetch TEU/CAPACITY using GraphQL and merge
        print("\n=== Step 7: Fetching TEU/CAPACITY for vessels via GraphQL API ===")
        imo_list = df_enriched['IMO'].dropna().astype(str).unique().tolist()
        df_teu = fetch_teu_by_imo_list(imo_list)

        df_enriched['IMO'] = df_enriched['IMO'].astype(str)
        df_teu['IMO'] = df_teu['IMO'].astype(str)
        df_enriched = df_enriched.merge(df_teu, on="IMO", how="left")

        cols = df_enriched.columns.tolist()
        if "TEU/CAPACITY" in cols and "SHIPNAME" in cols:
            cols.remove("TEU/CAPACITY")
            idx = cols.index("SHIPNAME") + 1
            cols.insert(idx, "TEU/CAPACITY")
            df_enriched = df_enriched[cols]

        # Arrange columns in desired order
        desired_cols = [
            "IMO", "SHIP_ID", "SHIPNAME", "TEU/CAPACITY",
            "LAST_PORT_ID", "LAST_PORT", "LAST_PORT_COUNTRY_CODE", "LAST_PORT_TIMESTAMP",
            "PORT_ID", "PORT_NAME", "LOAD_STATUS", "TIMESTAMP_UTC",
            "NEXT_PORT_ID", "NEXT_PORT", "NEXT_PORT_COUNTRY", "NEXT_PORT_TIMESTAMP"
        ]
        final_cols = [col for col in desired_cols if col in df_enriched.columns]
        df_enriched = df_enriched[final_cols]

        # Format all timestamp columns before saving
        for col in ["TIMESTAMP_UTC", "LAST_PORT_TIMESTAMP", "NEXT_PORT_TIMESTAMP"]:
            if col in df_enriched.columns:
                df_enriched[col] = pd.to_datetime(df_enriched[col], errors='coerce').dt.strftime('%d/%m/%Y %H:%M:%S')

        # Now save to CSV
        enriched_filename = f"{region.lower()}_portcalls_enriched_{datetime.utcnow().strftime('%Y%m%d_%H%M')}.csv"
        df_enriched.to_csv(enriched_filename, index=False)
        print(f"✅ Enriched port calls saved to {enriched_filename}")   

        agg_arrivals = aggregate_total_port_arrivals(df_enriched, output_filename=f'{region.lower()}_port_aggregation.csv')
        print(f"✅ Aggregation complete. Output: {region.lower()}_port_aggregation.csv")

    except KeyboardInterrupt:
        print("\nUser exited program.")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")

if __name__ == "__main__":
    main()